# Practica 2-4-2. Aggregations & GroupBy
- ## Juan Manuel González Ascencio -A00572003
- ## Hugo Ortiz Ponce - A01747206

(20 puntos) Generar DataFrame 

Genera un DataFrame con los datos del CSV proporcionado y despliega la estructura (forma),  columnas, índices, sus descriptores estadísticos básicos, así como el tipo de dato de cada variable. Verifica si hay valores nulos. 

In [4]:
import pandas as pd

In [5]:

df = pd.read_csv('teachingratings.csv')
print('Forma del DataFrame: ', df.shape)
print('Columnas: ', df.columns)
print('Índices: ', df.index)
print('Descriptores estadísticos básicos: ', df.describe())
print('Tipo de dato de cada variable: ', df.dtypes)
print('Valores nulos: ', df.isnull().sum())



Forma del DataFrame:  (463, 19)
Columnas:  Index(['minority', 'age', 'gender', 'credits', 'beauty', 'eval', 'division',
       'native', 'tenure', 'students', 'allstudents', 'prof', 'PrimaryLast',
       'vismin', 'female', 'single_credit', 'upper_division',
       'English_speaker', 'tenured_prof'],
      dtype='object')
Índices:  RangeIndex(start=0, stop=463, step=1)
Descriptores estadísticos básicos:                age        beauty        eval    students  allstudents  \
count  463.000000  4.630000e+02  463.000000  463.000000   463.000000   
mean    48.365011  6.271140e-08    3.998272   36.624190    55.177106   
std      9.802742  7.886477e-01    0.554866   45.018481    75.072800   
min     29.000000 -1.450494e+00    2.100000    5.000000     8.000000   
25%     42.000000 -6.562689e-01    3.600000   15.000000    19.000000   
50%     48.000000 -6.801430e-02    4.000000   23.000000    29.000000   
75%     57.000000  5.456024e-01    4.400000   40.000000    60.000000   
max     73.00000

Explorar contenido mediante simple agregación 

En el DataFrame (DF) realiza lo siguiente: 

- Mediante una operación aggregate, calcula el promedio de todas las columnas (numéricas) en el DF. 
- Realiza una operación de agregación por fila de manera que despliegues la suma por fila.
- Mediante una operación aggregate, calcula la desviación media absoluta de la variable beauty.
- Para la variable eval, calcula mediante operaciones de agregación el mínimo y máximo valor de item. 

In [17]:
#Mediante una operación aggregate, calcula el promedio de todas las columnas numéricas en el DF

df.head()

#dejamos unicamente las columnas age, beauty, eval, division, native, students, tenure, allstudents
dfnum = df[['age', 'eval','beauty', 'students', 'allstudents', 'prof']]
dfnum.aggregate('mean')


age            4.836501e+01
eval           3.998272e+00
beauty         6.271140e-08
students       3.662419e+01
allstudents    5.517711e+01
prof           4.543413e+01
dtype: float64

In [18]:
#Realiza una operación de agregación por fila de manera que despliegues la suma por fila.
dfnum.aggregate('sum', axis=1)

0      108.589916
1      251.989916
2      241.889916
3      241.689916
4      101.762268
          ...    
458    113.433396
459    168.611563
460    237.099420
461    156.943014
462    237.391822
Length: 463, dtype: float64

In [25]:
#Mediante una operación aggregate, calcula la desviación media absoluta de la variable beauty.

dfnum['beauty'].mad()

AttributeError: 'Series' object has no attribute 'mad'

In [24]:
#Para la variable eval, calcula mediante operaciones de agregación el mínimo y máximo valor de item. 
dfnum['eval'].aggregate(['min', 'max'])

min    2.1
max    5.0
Name: eval, dtype: float64

 

(50 puntos) Procesamiento de la información mediante agrupamiento:

- Agrupa según género para calcular el promedio de puntuaciones de belleza, así como su  desviación estándar y varianza. 
- Calcula el total de hombres y mujeres que son profesores titulares (tenure professor)
- Agrega una columna al resultado previo para desplegar el porcentaje de profesores titulares por género.
- Agrupa por género y calcula sus descriptores estadísticos básicos (conteo, promedio, percentiles 25, 50 y 75) de sus edades mediante alguna función de agregación.
- Define una función que filtre por suma de estudiantes > 100, agrupa por ID de profesor (prof)  y guarda en un nuevo DF. Finalmente, aplica una operación de filtrado para mostrar de este DF solo aquellos registros donde el profesor tenga en total más de 100 estudiantes.

In [22]:
#Agrupa según género para calcular el promedio de puntuaciones de belleza, así como su  desviación estándar y varianza
df.groupby('gender').aggregate({'beauty': ['mean', 'std', 'var']})

beauty                   
            mean      std       var
gender                             
female  0.116109  0.81781  0.668813
male   -0.084482  0.75713  0.573246

In [30]:
#Calcula el total de hombres y mujeres que son profesores titulares (tenure professor)
df.groupby('gender').aggregate({'tenured_prof': 'sum'})

,tenured_prof
gender,
female,145
male,216


In [37]:
#Agrega una columna al resultado previo para desplegar el porcentaje de profesores titulares por género.
df.groupby('gender').aggregate({'tenured_prof': 'sum', 'prof': 'count'}).assign(percentage = lambda x: x['tenured_prof']/x['prof']*100)

,tenured_prof,prof,percentage
gender,,,
female,145,195,74.358974
male,216,268,80.597015


In [40]:
# Agrupa por género y calcula sus descriptores estadísticos básicos (conteo, promedio, percentiles 25, 50 y 75) de sus edades mediante alguna función de agregación.

df.groupby('gender').aggregate({'age': ['describe']})

age                                                    
       describe                                                    
          count       mean       std   min   25%   50%    75%   max
gender                                                             
female    195.0  45.092308  8.532031  29.0  38.0  46.0  52.00  62.0
male      268.0  50.746269  9.993396  32.0  43.0  51.0  59.25  73.0

In [48]:
# Define una función que filtre por suma de estudiantes > 100, agrupa por ID de profesor (prof)  y guarda en un nuevo DF. Finalmente, aplica una operación de filtrado para mostrar de este DF solo aquellos registros donde el profesor tenga en total más de 100 estudiantes.

def filter_students(x):
    return x['students'].sum() > 100

df2 = df.groupby('prof').filter(filter_students)
df2[df2['students'] > 100]



,minority,age,gender,credits,beauty,eval,division,native,tenure,students,allstudents,prof,PrimaryLast,vismin,female,single_credit,upper_division,English_speaker,tenured_prof
8,no,51,male,more,-0.571984,3.2,upper,yes,yes,111,195,3,1,0,0,0,1,1,1
23,no,62,male,more,0.588569,4.2,upper,yes,yes,182,282,6,0,0,0,0,1,1,1
24,no,62,male,more,0.588569,4.4,upper,yes,yes,160,292,6,0,0,0,0,1,1,1
26,no,62,male,more,0.588569,4.3,upper,yes,yes,176,285,6,0,0,0,0,1,1,1
27,no,62,male,more,0.588569,4.6,upper,yes,yes,155,272,6,0,0,0,0,1,1,1
28,no,62,male,more,0.588569,4.8,upper,yes,yes,166,286,6,0,0,0,0,1,1,1
29,no,62,male,more,0.588569,4.3,upper,yes,yes,186,302,6,1,0,0,0,1,1,1
87,no,45,male,more,-0.062036,4.3,lower,yes,yes,120,161,16,1,0,0,0,0,1,1
100,no,48,male,more,-0.054344,4.2,lower,yes,no,113,140,18,1,0,0,0,0,1,0
171,no,39,male,more,0.576680,3.7,lower,yes,yes,154,316,33,0,0,0,0,0,1,1
